In [108]:
import pandas as pd
from fklearn.training.regression import xgb_regression_learner
from fklearn.training.pipeline import build_pipeline
import numpy as np
import collections
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

In [128]:
df_gestor = pd.read_csv('avaliacao_gestores.csv',sep=';')
df_gestor = df_gestor.astype(int)

In [114]:
df_gestor = df_gestor[df_gestor.n_atuacoes >= 3]

In [133]:
df_gestor.nota_var.describe(percentiles=[0.1,0.9])

count    44598.000000
mean         6.339118
std          7.936497
min        -13.000000
10%         -2.000000
50%          6.000000
90%         15.000000
max         30.000000
Name: nota_var, dtype: float64

In [137]:
df_gestor = df_gestor[(df_gestor.nota_var >= 15) | (df_gestor.nota_var <= -2)]

In [138]:
# SEPARACAO ENTRE TREINO E TESTE
dados_teste = df_gestor.sample(int(len(df_gestor)/6), replace = False)
index_teste = dados_teste.index
dados_treino = df_gestor.drop(index_teste)
dados_teste = dados_teste.reset_index(drop=True)
dados_treino = dados_treino.reset_index(drop=True)
print('dados:  ', len(df_gestor), '  treino:  ', len(dados_treino), '  teste:  ', len(dados_teste))

dados:   10152   treino:   8460   teste:   1692


# CONTRUCAO DOS MODELOS

In [142]:
reg2 = RandomForestRegressor(random_state=1, n_estimators=300)
#reg2 = linear_model.LinearRegression()

In [143]:
dt = dados_treino.drop('id_gestor',axis = 1).copy()
dty = dt.loc[:,'nota_var']
dtx = dt.drop('nota_var',axis=1)

In [144]:
reg2.fit(dtx,dty)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [145]:
dt['prediction'] = reg2.predict(dtx)

In [146]:
r2_score(dt.nota_var, dt.prediction)

0.3331542020846162

In [147]:
dtes = dados_teste.drop('id_gestor',axis=1)
dtesY = dtes.loc[:,'nota_var'].copy()
dtesX = dtes.drop('nota_var',axis=1)

In [148]:
dtes['prediction'] = reg2.predict(dtesX)

In [149]:
dtes.prediction.describe()

count    1692.000000
mean        7.788496
std         5.883979
min       -11.739444
25%         6.136929
50%         7.204235
75%        10.279153
max        28.138393
Name: prediction, dtype: float64

In [150]:
r2_score(dtes.nota_var, dtes.prediction)

-0.15127824578785165

In [139]:
AUXILIARY = ['id_gestor']
TARGET = ['nota_var']
FEATURES = set(df_gestor.columns) - set(AUXILIARY) - set(TARGET)

my_model = xgb_regression_learner(
    features=FEATURES,
    target=TARGET[0],
    prediction_column='prediction',
    extra_params={'seed': 139, 'nthread': 8},
)
my_learner = build_pipeline(my_model)

# TREINO
(prediction_function, treinado, logs) = my_learner(dados_treino)
# TESTE
testados = prediction_function(dados_teste)

In [140]:
r2_score(testados.nota_var, testados.prediction.tolist())

-0.03101072106268865

In [141]:
r2_score(treinado.nota_var.tolist(), treinado.prediction.tolist())

0.09953844118251132

In [107]:
df_gestor.corr()

,id_gestor,n_atuacoes,nota_var,primeira_viagem,teve_falta,formacao_pontos,idade,anos_trabalho,sexo
id_gestor,1.000000,-0.080036,0.006872,NaN,-0.816626,-0.177966,0.071169,-0.009655,0.491012
n_atuacoes,-0.080036,1.000000,-0.000874,NaN,0.054476,0.061109,0.023241,0.027932,-0.188374
nota_var,0.006872,-0.000874,1.000000,NaN,-0.012179,-0.028402,-0.007002,-0.024390,0.018526
primeira_viagem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
teve_falta,-0.816626,0.054476,-0.012179,NaN,1.000000,0.159551,-0.072288,0.025700,-0.434780
formacao_pontos,-0.177966,0.061109,-0.028402,NaN,0.159551,1.000000,-0.044456,-0.057997,-0.148808
idade,0.071169,0.023241,-0.007002,NaN,-0.072288,-0.044456,1.000000,0.421269,-0.089349
anos_trabalho,-0.009655,0.027932,-0.024390,NaN,0.025700,-0.057997,0.421269,1.000000,-0.100284
sexo,0.491012,-0.188374,0.018526,NaN,-0.434780,-0.148808,-0.089349,-0.100284,1.000000


# ESTATISTICA DESCRITIVA

In [121]:
df_gestor[df_gestor.sexo == 1].nota_var.mean()

6.212309264802512

In [122]:
df_gestor[df_gestor.sexo == 0].nota_var.mean()

6.035122786978869

In [154]:
df_gestor.idade.describe(percentiles=[0.1,0.9])

count    10152.000000
mean        46.869582
std          6.244007
min         26.000000
10%         38.000000
50%         48.000000
90%         54.000000
max         71.000000
Name: idade, dtype: float64

In [ ]:
df_gestor.idade.describe(percentiles=[0.1,0.9])

In [126]:
df_gestor[df_gestor.idade <= 46].nota_var.mean()

6.149603008775595

In [127]:
df_gestor[(df_gestor.idade > 50)].nota_var.mean()

6.043298969072165